In [1]:
!pip install bokeh

In [2]:
## import libraries
import os
import pandas as pd
import numpy as np

from bokeh import plotting as plt
from bokeh import models as md
from bokeh import io, layouts, palettes

plt.output_notebook()

In [3]:
# load data

def load_data(directory = '/content/drive/MyDrive/DataVisualization - DataFest'):

    inflation = pd.read_csv(os.path.join(directory,'data/inflation.csv'), encoding='utf8', parse_dates=['MONTH'])
    inflation.CPI_YOY = (inflation.CPI_YOY - 100)/100
    inflation.CPI_MOM = (inflation.CPI_MOM - 100)/100
    
    rates = pd.read_csv(os.path.join(directory,'data/ExchangeRatesGEL.csv'), parse_dates=['DATE'])
    rates = rates[rates.DATE >= '2016-12-01']
    rates['GEL_DEPR'] = rates.RATE.diff() >= 0
    
    inflation = inflation[inflation.MONTH >= rates.DATE.min()].sort_values(by = 'MONTH', ascending = True)

    rates['STREAM_ID'] = (~rates.GEL_DEPR).cumsum()
    rates['N_DEPRECIACIONS'] = rates.STREAM_ID.map(rates.groupby('STREAM_ID').DATE.count().to_dict()) - 1
    
    rates = rates.sort_values(by = 'DATE', ascending = True)
    nbg = pd.read_csv(os.path.join(directory,'data/nbg_open_market_operations.csv'), 
                      encoding = 'utf8',
                      parse_dates = ['DATE'],
                      )

    rates['NBG_SOLD'] = rates.DATE.map(nbg.set_index('DATE').AMT.to_dict()).fillna(0).cumsum()
    rates['ARTICLE'] = rates.DATE.map(nbg.set_index('DATE').RESP.to_dict()).fillna('')

    return rates, inflation

In [4]:
rates, inflation = load_data()

In [5]:
TOOLS = ['xbox_zoom','reset','save']

In [6]:
## Ex. Rate plot

# styling
BOX_COLOR = '#822A1A'
LINE_COLOR = '#173056'


# figure for line graph
f1 = plt.figure(x_axis_type="datetime", tools = TOOLS)

source = md.ColumnDataSource(data = rates)


# plot basic line
line_glyph = f1.line('DATE', 
                    'RATE', 
                    line_color=LINE_COLOR, 
                    source = source
                   )

# add hovertool
tooltips = [
    ('Date: ','@DATE{%F}'),
    ('USD/GEL: ','@RATE')
]

formatters = {
                '@DATE': 'datetime',                              
            }

hover_tool = md.HoverTool(tooltips = tooltips, 
                          formatters=formatters,
                          mode = 'vline')
f1.add_tools(hover_tool)


# highlight depreciaction periods
min_n_cons_days = 10
bad_period = rates.groupby('STREAM_ID').DATE.min().to_frame('PERIOD_START')
bad_period['PERIOD_END'] = bad_period.index.map(rates.groupby('STREAM_ID').DATE.max().to_dict())
bad_period['PERIOD_LEN'] = bad_period.index.map(rates.groupby('STREAM_ID').N_DEPRECIACIONS.first().to_dict())
bad_period = bad_period[bad_period.PERIOD_LEN >= min_n_cons_days]

for row_id in range(bad_period.shape[0]):
    row = bad_period.iloc[row_id]
    box = md.BoxAnnotation(bottom=0, 
                           top=99, 
                           left = row.PERIOD_START, 
                           right = row.PERIOD_END, 
                           fill_color=BOX_COLOR)
    f1.add_layout(box)


# plt.show(f1)

In [7]:
## Inflation plot

# styling
MOM_LINE_COLOR = '#173056'
YOY_LINE_COLOR = '#822A1A'



# figure for line graph
f2 = plt.figure(x_axis_type="datetime", tools = TOOLS)

source = md.ColumnDataSource(data = inflation)


# plot basic line
mom_infl_line_glyph = f2.line('MONTH', 
                    'CPI_MOM', 
                     color = MOM_LINE_COLOR,
                    source = source,
                        legend_label = 'MoM Infl.',
                   )

mom_infl_circle_glyph = f2.circle(x = 'MONTH', 
                         y = 'CPI_MOM',
                         fill_alpha = 0.1,
                         size = 7,         
                         color = MOM_LINE_COLOR,
                         source = source,
                         hover_alpha = 1,
                            legend_label = 'MoM Infl.',
                        )
yoy_infl_line_glyph = f2.line('MONTH', 
                    'CPI_YOY', 
                     color = YOY_LINE_COLOR,
                    source = source,
                        legend_label = 'YoY Infl.',
                   )

yoy_infl_circle_glyph = f2.circle(x = 'MONTH', 
                         y = 'CPI_YOY',
                         fill_alpha = 0.1,
                         size = 7,       
                         color = YOY_LINE_COLOR,
                         source = source,
                         hover_alpha = 1,
                            legend_label = 'YoY Infl.',
                        )


f2.legend.location = "top_right"
f2.legend.click_policy="hide"
f2.legend.label_text_font_size = '7pt'
f2.legend.background_fill_alpha = 0
f2.legend.glyph_height = f2.legend.glyph_width = 15
f2.legend.label_text_color = MOM_LINE_COLOR


# add hovertool
tooltips = [
    ('Date: ','@MONTH{%F}'),
    ('Inflation MoM: ','@CPI_MOM{0.0%}')
]

formatters = {
                '@MONTH': 'datetime',                              
            }

hover_tool = md.HoverTool(tooltips = tooltips, 
                          formatters=formatters,
                          mode = 'vline',
                          renderers = [mom_infl_line_glyph, mom_infl_circle_glyph],
                         )
f2.add_tools(hover_tool)


tooltips = [
    ('Date: ','@MONTH{%F}'),
    ('Inflation YoY: ','@CPI_YOY{0.0%}')
]

formatters = {
                '@MONTH': 'datetime',                              
            }

hover_tool = md.HoverTool(tooltips = tooltips, 
                          formatters=formatters,
                          mode = 'vline',
                          renderers = [yoy_infl_line_glyph, yoy_infl_circle_glyph],
                         )


f2.add_tools(hover_tool)



# plt.show(f2)

In [8]:
## NBG Injection plot
LINE_COLOR = '#173056'
AREA_COLOR = '#173056'

f3 = plt.figure(x_axis_type="datetime", tools = TOOLS)

source = md.ColumnDataSource(data = rates)


# plot basic area
line_glyph = f3.line('DATE', 
                    'NBG_SOLD', 
                    line_color=LINE_COLOR, 
                    source = source
                   )

area_glyph = f3.varea('DATE', 
                      y1 = 0,
                    y2 = 'NBG_SOLD', 
                    fill_color=AREA_COLOR, 
                    fill_alpha = 0.2,
                    source = source
                   )


# add hovertool
tooltips = """
            <div>
                <div>
                    <span style="font-weight:bold;">Cumulative intervention as of @DATE{%F}: @NBG_SOLD{0.0 a}</span>
                </div>
                    @ARTICLE
                </div>
           """

formatters = {
                '@DATE': 'datetime',
            }

hover_tool = md.HoverTool(tooltips = tooltips, 
                          formatters=formatters,
                          point_policy  = 'follow_mouse',
                          mode = 'vline',
                          renderers = [line_glyph],
                            )

f3.add_tools(hover_tool)

# plt.show(f3)

In [9]:
## layout

# configure sizes
f1.height = f2.height = f3.height = 200
f1.width = f2.width = f3.width = 900

f1.x_range = f2.x_range = f3.x_range


# set plot titles
f1.yaxis.axis_label = 'GEL/USD Daily Rate'
f2.yaxis.axis_label = 'Monthly Inflation'
f3.yaxis.axis_label = 'NBG Intervention'
f1.yaxis.axis_label_text_font_style = f2.yaxis.axis_label_text_font_style = f3.yaxis.axis_label_text_font_style = 'normal'


# format ticks
f3.xaxis.formatter= md.DatetimeTickFormatter(months = ['%b %Y'], days = ['%d %b %Y'])


# remove extra elements
f1.xaxis.visible = f2.xaxis.visible = False
f1.grid.visible = f2.grid.visible = f3.grid.visible = False

f1.yaxis.major_tick_line_color = None
f1.yaxis.minor_tick_line_color = None
f1.yaxis.major_label_text_font_size = '0pt'

f2.yaxis.major_tick_line_color = None
f2.yaxis.minor_tick_line_color = None
f2.yaxis.major_label_text_font_size = '0pt'

f3.yaxis.major_tick_line_color = None
f3.yaxis.minor_tick_line_color = None
f3.yaxis.major_label_text_font_size = '0pt'


# styling
f1.yaxis.axis_label_text_color=f2.yaxis.axis_label_text_color=f3.yaxis.axis_label_text_color='#333333'
f1.background_fill_color = f2.background_fill_color = f3.background_fill_color = '#C4C4C4'
f1.border_fill_color = f2.border_fill_color = f3.border_fill_color = '#C4C4C4'
f1.background_fill_alpha = f2.background_fill_alpha = f3.background_fill_alpha = 0.1
f1.border_fill_alpha = f2.border_fill_alpha = f3.border_fill_alpha = 0.1
f1.outline_line_color = f2.outline_line_color = f3.outline_line_color = '#333333'
f1.yaxis.axis_line_color=f2.yaxis.axis_line_color=f3.yaxis.axis_line_color='#333333'
f3.xaxis.major_label_text_color = '#333333'


lay = layouts.gridplot(children = [[f1], [f2], [f3]])

plt.show(lay)